#PRAKTIKUM 8
# Evaluasi Beberapa Algoritma Klasifikasi

Tujuan :
Setelah mengikuti praktikum ini praktikan
1.   Memahasi Evaluasi beberapa metrik untuk mengukur kinerja klasifikasi
2.   Mahasiswa dapat memahami confisin matrik dan perananya untuk evaluasi klasifikasi
3. Memehami dan mampu menggunakan evaluasi k-fold








Nama  : Ahmad Zaki

NIM   : 225150201111025

Kelas : PPM-D

##Dasar Teori

Metode Validasi

Metode validasi digunakan untuk menguji sejauh mana akurasi klasifikasi dari
metode klasifikasi yang dikembangkan. Metode ini dikenal dengan cross-validation
dimana data masukan dipecah menjadi data laith dan data uji. Pada metode ini terdapat beberapa metode, metode yang paling sederhana adalah adalah 2-fold cros-validation dimana data dipisah secara acak menjadi dalam latih dan data uji. Pengembangan dari 2-fold cross-validation adlah K-fold cross-validation dimana data dipisah secara acak ke dalam K subset. Untuk pelatihan subset K-1 dipilih dan sisanya digunakan untuk pengujian. Proses ini diulangi sampai semua subset digunakan untuk pengujian. Versi lain dari K-fold cross-validation dalah Leave-one out cross-validation (LOOCV) diman K sama dengan jumlah sampel yakni masing-masing sampel digunakan untuk testing dan sisanya digunakan untuk pelatihan. Proses ini diulangi sampai semua sampel diuji.

**Tolak Ukur Kinerja**

Keluaran suatu klasifikasi dapat dikatagorikan dalam 4 tipe sebagaimana
dalam matrik confusion . jika keluaran klasifikasi konsisten dengan kebenaran, maka hal ini disebut True Positif (TP) atau True Negative (TN), tergantung pada nilai aktualnya. Sedangkan yang lain disebut False Positif (FP) atau False Negative (FN). Dalam domain masalah yang berbeda, FP dan TN depreciated dan diboboti secara berbeda. Sebagai contoh, FP lebih toleran daripada FN dalam diagnose penyakit manusia.

#Tahapan validasi metode klasifikasi


##1. Tentukan metode klasifikasi yang akan dievaluasi

In [198]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [199]:
models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

# evaluate each model in turn
results = []
names = []

##2. Baca dataset
Ambil dataset yang akan dieavaluasi. Untuk praktikum ini dievaluasi 3 dataset sebagai berikut :
- dataR2
- iris data
- breast cancer

Untuk praktek dataset dataR2 digunakan sebagai contoh sedangkan dua data berikunya akan digunakan sebagai latihan.

In [200]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv

--2024-05-03 07:59:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘dataR2.csv.5’

dataR2.csv.5            [ <=>                ]   7.49K  --.-KB/s    in 0s      

2024-05-03 07:59:56 (107 MB/s) - ‘dataR2.csv.5’ saved [7665]



In [201]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-05-03 07:59:56--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.5’

iris.csv.5          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-05-03 07:59:56 (28.7 MB/s) - ‘iris.csv.5’ saved [3975/3975]



In [202]:
! wget https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv

--2024-05-03 07:59:56--  https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3664 (3.6K) [text/plain]
Saving to: ‘breast_cancer.csv.5’

breast_cancer.csv.5 100%[===================>]   3.58K  --.-KB/s    in 0s      

2024-05-03 07:59:56 (49.2 MB/s) - ‘breast_cancer.csv.5’ saved [3664/3664]



##3. Tampilkan dataset

In [203]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('dataR2.csv')
dataset

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1
...,...,...,...,...,...,...,...,...,...,...
111,45,26.850000,92,3.330,0.755688,54.6800,12.100000,10.96000,268.230,2
112,62,26.840000,100,4.530,1.117400,12.4500,21.420000,7.32000,330.160,2
113,65,32.050000,97,5.730,1.370998,61.4800,22.540000,10.33000,314.050,2
114,72,25.590000,82,2.820,0.570392,24.9600,33.750000,3.27000,392.460,2


##4. Membuat Set Data Validasi
Tujuan evaluasi adalah memperkirakan keakuratan model terbaik pada data tak terlihat dengan mengevaluasinya pada data tak terlihat aktual. Evaluasi ini dilakukan dengan cara membagi dataset menjadi dua bagian yakni sebagian digunakan sebagai data latih dan sebagian yang lain digunakan sebagai data uji. Beberapa praktikum sebelumnya telah dipraktekkan dengan menggunakan **metode train_test_split : X_train, X_validation, Y_train, Y_validation = train_test_split(X,y,test_size=0.20, random_state=1)**


Sebelum membuat data latih dan data uji. Langkah pertama pisahkan antara data attribute dan data label suatu dataset. Karena datasaet data2R terdiri dari 9 attribut dan 1 label.  Maka kita catat X sebagai data dan Y sebagai label.

In [204]:
from sklearn.model_selection import train_test_split
array = dataset.values
X = array[:,0:9]
y = array[:,9]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

##K-Fold Cross Validation
k-Fold Cross-Validation Validasi silang adalah prosedur pengambilan sampel ulang yang digunakan untuk mengevaluasi model pembelajaran mesin pada sampel data terbatas. Prosedur ini memiliki parameter tunggal yang disebut k yang mengacu pada jumlah grup yang akan dipecah menjadi sampel data tertentu. Karena itu, prosedur ini sering disebut k-fold cross-validation. Ketika nilai spesifik untuk k dipilih, nilai tersebut dapat digunakan sebagai pengganti k dalam referensi ke model, seperti k = 10 menjadi validasi silang 10 kali lipat.

Validasi silang terutama digunakan dalam pembelajaran mesin terapan untuk memperkirakan kemampuan model pembelajaran mesin pada data yang tidak terlihat. Yaitu, menggunakan sampel terbatas untuk memperkirakan bagaimana performa model secara umum saat digunakan untuk membuat prediksi pada data yang tidak digunakan selama pelatihan model.

Prosedur umumnya adalah sebagai berikut:
- Acak kumpulan data berdasarkan id
- Pisahkan kumpulan data menjadi kelompok k
- Untuk setiap grup unik:
  - Ambil kelompok sebagai kelompok data uji
  - Ambil kelompok yang tersisa sebagai kumpulan data pelatihan
  - Catat hasil pengujian masing-masing group

K-Fold ini digunakan untuk validasi model yang digunakan. Evaluasi dilakukan pada dataset latih. Pada praktikum ini digunakan k=10, berarti akan membagi dataset latih menjadi 10 bagian. Satu bagian digunakan untuk testing dan sisanya sebagai data latih.

In [205]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
for name, model in models:
 kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
 results.append(cv_results)
 names.append(name)
 print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

KNN: 0.487778 (0.143540)
NB: 0.662222 (0.125629)
SVM: 0.465556 (0.047778)


##4. Buat Prediksi

Diantara metode klasifikasi yang telah dievaluasi, Kita pilih algoritma yang mempunyai kinerja paling baik yang akan digunakan untuk membuat metode prediksi Prediksi dilakukan dengan mengevaluasi dataset evaluasi menggunakan metode klasifikasi yang telah terpilih (dalam hal ini metede terbaik adalah SVM). Dengan demikian metode SVM kita gunakan sebagai prediktor

In [206]:
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

# 5. Evaluasi Prediksi
Kami dapat mengevaluasi prediksi dengan membandingkannya dengan hasil yang diharapkan dalam set validasi, lalu menghitung akurasi klasifikasi, serta matriks kebingungan dan laporan klasifikasi.

In [207]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [208]:
from sklearn.metrics import classification_report
print(classification_report(Y_validation, predictions))

              precision    recall  f1-score   support

         1.0       0.25      1.00      0.40         6
         2.0       0.00      0.00      0.00        18

    accuracy                           0.25        24
   macro avg       0.12      0.50      0.20        24
weighted avg       0.06      0.25      0.10        24



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Tugas :

1. Lakukan perbandingan klasifikasi antara SVM, Naive Bayes, KNN untuk dataset iris dan breast cancer
2. Pada kedua dataset lakukan proses normaslisai data dan preproses data untuk menengani data yang hilang jika ada.

In [209]:
iris = pd.read_csv('iris.csv')
iris.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [210]:
breast = pd.read_csv('breast_cancer.csv')
breast.head()

,id,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nucklei,bland_chromatin,normal_nucleoli,mitoses,diagnosis
0,1000025,5.0,1.0,NaN,1.0,NaN,1.0,3.0,1.0,1.0,benign
1,1002945,5.0,4.0,4.0,5.0,7.0,NaN,3.0,NaN,1.0,benign
2,1015425,3.0,1.0,1.0,1.0,NaN,2.0,3.0,1.0,1.0,benign
3,1016277,6.0,8.0,8.0,1.0,3.0,NaN,3.0,7.0,1.0,benign
4,1017023,4.0,1.0,1.0,3.0,NaN,1.0,3.0,1.0,NaN,benign


In [211]:
def imputasi(df_input):
  list_columns = df_input.columns
  class_column = list_columns[-1]
  for column in list_columns[:-1]:
    df_input[column] = df_input[column].fillna(df_input.groupby(class_column)[column].transform('mean'))
  return df_input

In [212]:
def cetak_rentang(df_input):
  list_fitur = df_input.columns[:-1]
  for fitur in list_fitur:
    max = df_input[fitur].max()
    min = df_input[fitur].min()
    print("Rentang fitur", fitur, " adalah ", max-min)

In [213]:
def minmax(df_input):
  df_minmax = df_input.copy()
  list_fitur = df_minmax.columns[:-1]
  for fitur in list_fitur:
    max = df_minmax[fitur].max()
    min = df_minmax[fitur].min()
    df_minmax[fitur] = (df_minmax[fitur]-min) / (max - min)
  return df_minmax

In [214]:
iris = iris.replace(0.0, np.NaN)

In [215]:
pd.isna(iris).values.any()

False

In [216]:
iris.isnull().values.any()

False

In [217]:
cetak_rentang(iris)

Rentang fitur sepal.length  adalah  3.6000000000000005
Rentang fitur sepal.width  adalah  2.4000000000000004
Rentang fitur petal.length  adalah  5.9
Rentang fitur petal.width  adalah  2.4


In [218]:
iris_normal = minmax(iris)

In [219]:
cetak_rentang(iris_normal)

Rentang fitur sepal.length  adalah  1.0
Rentang fitur sepal.width  adalah  1.0
Rentang fitur petal.length  adalah  1.0
Rentang fitur petal.width  adalah  1.0


In [220]:
iris_normal.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,0.222222,0.625000,0.067797,0.041667,Setosa
1,0.166667,0.416667,0.067797,0.041667,Setosa
2,0.111111,0.500000,0.050847,0.041667,Setosa
3,0.083333,0.458333,0.084746,0.041667,Setosa
4,0.194444,0.666667,0.067797,0.041667,Setosa


In [221]:
array_iris = iris_normal.values
X_iris = array_iris[:,0:4]
y_iris = array_iris[:,4]
X_iris_train, X_iris_validation, Y_iris_train, Y_iris_validation = train_test_split(X_iris, y_iris, test_size = 0.2, random_state = 1, shuffle=True)

In [222]:
for name, model in models:
  kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle=True)
  cv_results = cross_val_score(model, X_iris_train, Y_iris_train, cv=kfold, scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

KNN: 0.950000 (0.040825)
NB: 0.950000 (0.055277)
SVM: 0.941667 (0.053359)


In [223]:
model = SVC(gamma='auto')
svc = model.fit(X_iris_train, Y_iris_train)
prediction = svc.predict(X_iris_validation)

In [224]:
X_iris, y_iris = make_classification(n_samples=1000, n_features=20, n_informative
=15, n_redundant=5, random_state=1)
from sklearn.metrics import classification_report
print(classification_report(Y_iris_validation, prediction))

              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00        11
  Versicolor       1.00      0.92      0.96        13
   Virginica       0.86      1.00      0.92         6

    accuracy                           0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30



In [225]:
print(breast.isnull().values.any())
print(pd.isna(breast).values.any())

True
True


In [226]:
breast = breast.replace(0.0, np.NaN)

In [227]:
breast_imputasi = imputasi(breast)
breast_imputasi.head()

,id,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nucklei,bland_chromatin,normal_nucleoli,mitoses,diagnosis
0,1000025,5.0,1.0,1.636364,1.0,2.152174,1.000000,3.0,1.000000,1.000000,benign
1,1002945,5.0,4.0,4.000000,5.0,7.000000,1.372093,3.0,1.363636,1.000000,benign
2,1015425,3.0,1.0,1.000000,1.0,2.152174,2.000000,3.0,1.000000,1.000000,benign
3,1016277,6.0,8.0,8.000000,1.0,3.000000,1.372093,3.0,7.000000,1.000000,benign
4,1017023,4.0,1.0,1.000000,3.0,2.152174,1.000000,3.0,1.000000,1.113636,benign


In [228]:
print(breast.isnull().values.any())
print(pd.isna(breast).values.any())

False
False


In [229]:
cetak_rentang(breast_imputasi)

Rentang fitur id  adalah  170394
Rentang fitur clump_thickness  adalah  9.0
Rentang fitur cell_size  adalah  9.0
Rentang fitur cell_shape  adalah  9.0
Rentang fitur marginal_adhesion  adalah  9.0
Rentang fitur epithelial_cell_size  adalah  9.0
Rentang fitur bare_nucklei  adalah  9.0
Rentang fitur bland_chromatin  adalah  8.0
Rentang fitur normal_nucleoli  adalah  9.0
Rentang fitur mitoses  adalah  9.0


In [230]:
breast_normal = minmax(breast_imputasi)

In [231]:
cetak_rentang(breast_normal)

Rentang fitur id  adalah  1.0
Rentang fitur clump_thickness  adalah  1.0
Rentang fitur cell_size  adalah  1.0
Rentang fitur cell_shape  adalah  1.0
Rentang fitur marginal_adhesion  adalah  1.0
Rentang fitur epithelial_cell_size  adalah  1.0
Rentang fitur bare_nucklei  adalah  1.0
Rentang fitur bland_chromatin  adalah  1.0
Rentang fitur normal_nucleoli  adalah  1.0
Rentang fitur mitoses  adalah  1.0


In [232]:
breast_normal.head()

,id,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nucklei,bland_chromatin,normal_nucleoli,mitoses,diagnosis
0,0.000000,0.444444,0.000000,0.070707,0.000000,0.128019,0.000000,0.25,0.000000,0.000000,benign
1,0.017137,0.444444,0.333333,0.333333,0.444444,0.666667,0.041344,0.25,0.040404,0.000000,benign
2,0.090379,0.222222,0.000000,0.000000,0.000000,0.128019,0.111111,0.25,0.000000,0.000000,benign
3,0.095379,0.555556,0.777778,0.777778,0.000000,0.222222,0.041344,0.25,0.666667,0.000000,benign
4,0.099757,0.333333,0.000000,0.000000,0.222222,0.128019,0.000000,0.25,0.000000,0.012626,benign


In [233]:
array_breast = breast_normal.values
X_breast = array_breast[:,0:10]
y_breast = array_breast[:,10]
X_breast_train, X_breast_validation, Y_breast_train, Y_breast_validation = train_test_split(X_breast, y_breast, test_size = 0.2, random_state = 1, shuffle=True)

In [234]:
for name, model in models:
  kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle=True)
  cv_results = cross_val_score(model, X_breast_train, Y_breast_train, cv=kfold, scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

KNN: 0.925000 (0.100000)
NB: 0.900000 (0.108972)
SVM: 0.950000 (0.082916)


In [235]:
model = SVC(gamma='auto')
svc = model.fit(X_breast_train, Y_breast_train)
prediction = svc.predict(X_breast_validation)

In [236]:
X_breast, y_breast = make_classification(n_samples=1000, n_features=20, n_informative
=15, n_redundant=5, random_state=1)
from sklearn.metrics import classification_report
print(classification_report(Y_breast_validation, prediction))

              precision    recall  f1-score   support

      benign       1.00      1.00      1.00         9
   malignant       1.00      1.00      1.00        11

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



Pada praktikum ini, saya membandingkan klasifikasi menggunakan algoritma Support Vector Machine (SVM), Naive Bayes (NB), dan K-Nearest Neighbors (KNN) pada dataset iris dan breast cancer, sebelum masuk ke tahap evaluasi model saya melakukan preprocessing data dengan mengimputasi dan normalisasi. Pada kedua dataset, saya menggunakan imputasi berbasis grup untuk mengganti nilai yang hilang dan normalisasi Min-Max untuk menstandarkan range nilai.

Masuk ke evaluasi model, terdapat perbedaan yang menarik dalam hasil evaluasi. Pada dataset iris, SVM menunjukkan akurasi sebesar 97%, dengan presisi dan recall yang seimbang untuk setiap kelas. Naive Bayes dan KNN juga memberikan hasil yang cukup baik dengan akurasi sekitar 95%, meskipun sedikit lebih rendah dari SVM.

Namun, pada dataset breast cancer, SVM menonjol dengan akurasi sempurna sebesar 100%, dengan presisi, recall, dan F1-score semua bernilai 100% untuk setiap kelas. Sementara itu, Naive Bayes memberikan akurasi 90% dengan presisi dan recall yang sempurna, tetapi F1-score yang sedikit lebih rendah. KNN juga memberikan hasil yang baik dengan akurasi 92.5%, tetapi presisi dan F1-score sedikit lebih rendah dibandingkan SVM.

Ahmad Zaki - 225150201111025 :)